## 집들이에 태깅된 상품정보 스크래핑

해당 코드는 100개에 한번씩 데이터를 저장하는 방식으로 짜여져 있습니다.

코랩의 경우 구글드라이브와 연동해야 합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

구글 드라이브에 파일을 저장할 폴더를 새로 만들고 그 경로를 `path`에 넣어주세요

예) '/content/drive/MyDrive/폴더명'

In [ ]:
path = '경로' ##### 저장할 폴더의 경로. 슬래쉬(/)로 끝난다면 마지막 슬래쉬를 지워주세요.

상품 상세페이지가 적혀있는 파일을 불러옵니다

In [ ]:
import pandas as pd
df = pd.read_csv('prd_urls.csv')
df = df.drop('Unnamed: 0', axis = 1)
df = df.reset_index(drop = True)
df

본인이 맡은부분을 start와 end에 넣어주세요.

`김민지`  [11300:45300]

`김예원`  [45300:79300]

`박유정`  [79300:113300]

`이소현`  [113300:147300]

`이지혜`  [147300:181300]

`최수연`  [181300:215300]

`송나현`  [:11300], [215300:]

In [ ]:
start = ##### 자신이 맡은 부분 시작점
end = ##### 자신이 맡은 부분 종료지점

In [ ]:
df_prd = pd.DataFrame(columns = {'브랜드명', '상품명', '상품가격', '카테고리', '리뷰수', '스크랩수', '스타일링샷수'})
df_prd = pd.concat([df[start:end], df_prd], axis = 1)
df_prd = df_prd[['id', '상품url', '브랜드명', '상품명', '상품가격', '카테고리', '리뷰수', '스크랩수', '스타일링샷수']]
df_prd

아래 코드로 스크래핑을 시작합니다. 

In [ ]:
# 코랩에서 드라이브 오류날 경우 이 셀을 실행시킵니다.

# # install chromium, its driver, and selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium

In [ ]:
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.display import display
import warnings

In [ ]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(3)


for idx in df_prd.index:


    # Selenium을 통해 실제 크롬창에서 접속한 것과 동일하게 해당 URL내용을 가져옴
    driver.get(df_prd['상품url'][idx])
    # 소스를 읽어 BeautifulSoup를 이용해 파싱
    html = driver.page_source
    soup = BS(html, 'html.parser')

    categories = soup.select('body > div.page > div > div > div.production-selling > div.production-selling-overview.container > nav > ol')
    products = soup.select('body > div.page > div > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header')
    price = soup.select('body > div.page > div > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header > div.production-selling-header__content > div.production-selling-header__price > span.production-selling-header__price__price-wrap > span.production-selling-header__price__price > span.number')
    counts = soup.find_all('span', 'count')


    cates = []
    for cts in categories[0].find_all('a'):
        cates.append(cts.text)

    
    df_prd['카테고리'][idx] = cates
    df_prd['브랜드명'][idx] = products[0].find('a').text
    df_prd['상품명'][idx] = products[0].find('span').text
    
    # 스크랩 건수가 있는 경우
    if counts[0].text:
        df_prd['스크랩수'][idx] = counts[0].text
        
    # 스타일링샷 건수가 있는 경우
    if counts[1].text:
        df_prd['스타일링샷수'][idx] = counts[1].text

    # 가격이 있는 경우 (미입점된 상품의 경우 없음)    
    if price:
        df_prd['상품가격'][idx] = price[0].text

    # 리뷰가 있는지 확인
    try:
        review = soup.find_all('a', 'production-selling-navigation__item')[1]
    except:
        pass
    else:
        if review.find('span'):
            df_prd['리뷰수'][idx] = review.find('span').text

    # 데이터 100건이 쌓일 때마다 저장
    if idx % 100 == 99:
        print(idx + 1)
        df_prd.loc[start:idx+1].to_csv(path + f'/prd_data_{start}_{idx}.csv') ##### 경로부분에 파일을 저장할 경로 넣기

driver.close()

df_prd.to_csv(path + f'/prd_data_{start}_{idx}.csv')


- 만약 코드를 돌리다 끊어질 경우
    - 괜찮아요! 100개 단위로 저장되어 있으니 가장 마지막에 저장된 파일 제목에 힌트가 있습니다.
    - 마지막 파일의 파일명이 `prd_data_0_399.csv` 라면 `start`에 400을 입력하고 다시 돌리면 됩니다.
    - 돌리기 전에 마지막 파일을 제외하고 다 삭제해주세요! 파일들을 모아서 병합해야하는데, 이전에 파일들을 모두 가지고 있으면 중복값이 생깁니다.
    - 400부터 다시 돌리면 `prd_data_400_799.csv` 이런식으로 바로 파일명이 연결될 거예요 이렇게 연결되는 파일을 모아서 주시면 됩니다.
    
- 돌아가다 끊긴 게 아니라 뭔가 오류가 생긴 경우
    - 바로 저에게 연락주세요! 1만개까지는 생기는 오류를 잡았는데 또 어떤 오류가 나타날지 모르겠어요 😢
    
**데이터가 많아서 오래 걸릴텐데 함께 돌려주시는 여러분 정말 감사해요♥️**

**그럼 잘 부탁드립니다☺️**